In [128]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Flatten
from keras.layers import Embedding

In [129]:
import numpy as np

In [205]:
# Load data

def load_list(filename):
    vocabulary = []
    with open(filename, 'r') as f:
        for l in f:
            vocabulary.append(l.strip())
    return np.asarray(vocabulary)

def load_csv(filename):
    import csv
    
    sentence = []
    with open('./sentence_data/pos_related.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            sentence.append(row)
    return np.asarray(sentence).flatten()
    
pos_related = load_csv('./sentence_data/pos_related.csv')
neg_related = load_csv('./sentence_data/neg_related.csv')
pos_unrelated = load_csv('./sentence_data/pos_unrelated.csv')
neg_unrelated = load_csv('./sentence_data/neg_unrelated.csv')

In [206]:
print(pos_related.shape)
print(neg_related.shape)
print(pos_unrelated.shape)
print(neg_unrelated.shape)

(466,)
(466,)
(466,)
(466,)


In [207]:
# Stack the data

related_set = np.hstack((pos_related, neg_related))
print(related_set.shape)
y_related = np.ones(related_set.shape)
print(y_related.shape)

unrelated_set = np.hstack((pos_unrelated, neg_unrelated))
print(unrelated_set.shape)
y_unrelated = np.zeros(unrelated_set.shape)
print(y_unrelated.shape)

X_stack = np.hstack((related_set, unrelated_set))
y_stack = np.hstack((y_related, y_unrelated))

X_stack.shape

(932,)
(932,)
(932,)
(932,)


(1864,)

In [281]:
from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
tf_vectorizer = CountVectorizer(lowercase=True, max_df=1.0, min_df=5, binary=False, token_pattern=token)
tf_vectorizer.set_params(ngram_range=(1,1))

# X_vectorized = tf_vectorizer.fit_transform(X_stack)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_stack, test_size=0.33, random_state=42)

tf_vectorizer.fit(X_stack)

word_dict = tf_vectorizer.vocabulary_

In [282]:
word_dict

{'10/10': 0,
 '15': 1,
 '2': 2,
 '3': 3,
 '5': 4,
 "50's": 5,
 '7': 6,
 '9': 7,
 '99': 8,
 'a': 9,
 'about': 10,
 'absolute': 11,
 'absolutely': 12,
 'acted': 13,
 'acting': 14,
 'action': 15,
 'actor': 16,
 'actually': 17,
 'adaptation': 18,
 'admit': 19,
 'after': 20,
 'again': 21,
 'against': 22,
 'age': 23,
 'all': 24,
 'almost': 25,
 'along': 26,
 'already': 27,
 'also': 28,
 'although': 29,
 'altogether': 30,
 'always': 31,
 'am': 32,
 'amazing': 33,
 'american': 34,
 'among': 35,
 'an': 36,
 'and': 37,
 'andrew': 38,
 'animation': 39,
 'ann': 40,
 'any': 41,
 'anyone': 42,
 'anything': 43,
 'anyway': 44,
 'are': 45,
 'around': 46,
 'as': 47,
 'at': 48,
 'audience': 49,
 'awesome': 50,
 'back': 51,
 'bad': 52,
 'basically': 53,
 'be': 54,
 'beautiful': 55,
 'beautifully': 56,
 'because': 57,
 'been': 58,
 'before': 59,
 'being': 60,
 'believable': 61,
 'best': 62,
 'better': 63,
 'between': 64,
 'big': 65,
 'bit': 66,
 'black': 67,
 'blaise': 68,
 'blood': 69,
 'bold': 70,
 'bond

In [283]:
from textblob import TextBlob
import re

token_pattern = re.compile(token)
X = []
i=0
for sentence in X_stack:
    split = token_pattern.findall(sentence)
    seq = []
    for word in split:
        try:
            seq.append(word_dict[word])
        except KeyError:
            continue
    X.append(seq)
    
X = np.asarray(X)

print(X.shape)

(1864,)


In [196]:
y_stack[260]

1.0

In [285]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_stack, test_size=0.33, random_state=42)

In [286]:
print('Pad sequences (samples x time)')
maxlen=100
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (1248, 100)
x_test shape: (616, 100)


In [284]:
dict_len = len(tf_vectorizer.get_feature_names())
batch_size = 32
print(dict_len)

605


In [287]:
print('Build model...')
model = Sequential()
model.add(Embedding(dict_len, 500))
model.add(LSTM(output_dim=hidden_neurons, input_dim=500))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam')   

Build model...


C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(None, 500..., units=10)`
  after removing the cwd from sys.path.


In [288]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 500)         302500    
_________________________________________________________________
lstm_21 (LSTM)               (None, 10)                20440     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 322,951
Trainable params: 322,951
Non-trainable params: 0
_________________________________________________________________


In [294]:
from keras.callbacks import TensorBoard
from time import time

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [297]:

print('Train...')
hist = model.fit(x_train, y_train, epochs=50, verbose=1, callbacks=[tensorboard])


Train...
Epoch 1/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.5952
Epoch 2/50
1248/1248 [==============================] - 5s 4ms/step - loss: 0.5956
Epoch 3/50
1248/1248 [==============================] - 6s 4ms/step - loss: 0.5946
Epoch 4/50
1248/1248 [==============================] - 6s 5ms/step - loss: 0.5947
Epoch 5/50
1248/1248 [==============================] - 7s 5ms/step - loss: 0.5942
Epoch 6/50
1248/1248 [==============================] - 7s 5ms/step - loss: 0.5945
Epoch 7/50
1248/1248 [==============================] - 7s 6ms/step - loss: 0.5943
Epoch 8/50
1248/1248 [==============================] - 7s 6ms/step - loss: 0.5944
Epoch 9/50
1248/1248 [==============================] - 8s 6ms/step - loss: 0.5939
Epoch 10/50
1248/1248 [==============================] - 7s 6ms/step - loss: 0.5947
Epoch 11/50
1248/1248 [==============================] - 7s 5ms/step - loss: 0.5952
Epoch 12/50
1248/1248 [==============================] - 7s 5ms/step - loss:

In [268]:

y_predict = model.predict(x_test)

y = []

for pred in y_predict:
    if pred > 0.5:
        y.append(1)
    else:
        y.append(0)

        
np.sum(y_test == y)/len(y)

0.21915584415584416

In [228]:
in_neurons = voc 

hidden_neurons = 10

In [12]:
# define the model
model = Sequential()
# model.add(Embedding(voc, 32, input_length=max_length))
# model.add(Flatten())
model.add(LSTM(output_dim=hidden_neurons, input_dim=in_neurons))

model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'], class_mode="binary")
# summarize the model
print(model.summary())

C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(None, 368..., units=10)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                147840    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 147,851
Trainable params: 147,851
Non-trainable params: 0
_________________________________________________________________
None
